# 手写数字识别 CNN (使用 MPS 加速)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import time

# 检查 CUDA 和 MPS 设备是否可用，优先使用 CUDA，其次 MPS，最后使用 CPU
# 检查 CUDA 设备是否可用
if torch.cuda.is_available():
    device = torch.device(f"cuda:{torch.cuda.current_device()}")
# 检查 MPS 设备是否可用
elif torch.backends.mps.is_available():
    device = torch.device("mps")
# 若 CUDA 和 MPS 都不可用，使用 CPU
else:
    device = torch.device("cpu")
print(f"使用设备: {device}")

In [ ]:
# 定义CNN模型
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # 第一层卷积: 1个输入通道, 32个输出通道,  kernel_size=3
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        # 第二层卷积: 32个输入通道, 64个输出通道, kernel_size=3
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # 池化层: kernel_size=2, stride=2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # 全连接层: 7*7*64 -> 128
        self.fc1 = nn.Linear(7*7*64, 128)
        # 全连接层: 128 -> 10 (10个类别)
        self.fc2 = nn.Linear(128, 10)
        # Dropout层
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # 第一层卷积 + ReLU + 池化
        x = self.pool(F.relu(self.conv1(x)))
        # 第二层卷积 + ReLU + 池化
        x = self.pool(F.relu(self.conv2(x)))
        # 展平
        x = x.view(-1, 7*7*64)
        # 全连接层 + ReLU + Dropout
        x = self.dropout(F.relu(self.fc1(x)))
        # 输出层
        x = self.fc2(x)
        return x

In [ ]:
# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 加载MNIST数据集
train_dataset = datasets.MNIST(
    root='./data', train=True, download=True, transform=transform
)

test_dataset = datasets.MNIST(
    root='./data', train=False, download=True, transform=transform
)

# 创建数据加载器
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# 初始化模型、损失函数和优化器
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 训练模型
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 100 == 99:
            print(f'[{epoch}, {batch_idx+1}] loss: {running_loss/100:.3f}')
            running_loss = 0.0

# 测试模型
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)')

In [ ]:
# 开始训练和测试
epochs = 5
start_time = time.time()

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)

end_time = time.time()
print(f'训练和测试总用时: {end_time - start_time:.2f}秒')

# 保存模型
os.makedirs('model', exist_ok=True)  # 确保model目录存在
torch.save(model.state_dict(), 'model/mnist_cnn_model.pth')
print('模型已保存为 model/mnist_cnn_model.pth')

## 模型预测演示

In [ ]:
import matplotlib.pyplot as plt

# # 设置字体以支持中文
# # 以 Windows 为例，使用 'SimHei'
# plt.rcParams['font.sans-serif'] = ['PingFang SC']
# # 解决负号 '-' 显示为方块的问题
# plt.rcParams['axes.unicode_minus'] = False


# import matplotlib as mpl
# print(mpl.get_cachedir())
# print(mpl.matplotlib_fname())

# 加载测试图像并显示
def visualize_prediction(model, device, test_loader, num_samples=5):
    model.eval()
    images, labels = next(iter(test_loader))
    images, labels = images[:num_samples].to(device), labels[:num_samples].to(device)
    
    with torch.no_grad():
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
    
    fig, axes = plt.subplots(1, num_samples, figsize=(12, 3))
    for i in range(num_samples):
        ax = axes[i]
        ax.imshow(images[i].cpu().numpy().squeeze(), cmap='gray')
        ax.set_title(f'真实: {labels[i].item()}, 预测: {predicted[i].item()}')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# 可视化预测结果
visualize_prediction(model, device, test_loader)

In [ ]:
# `mnist_cnn_model.pth` 通常是 PyTorch 保存的模型文件，下面展示如何加载和使用这个文件。

import torch
from torchvision import datasets, transforms
import torch.nn as nn

# 定义与保存模型时相同的模型结构
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # 第一层卷积: 1个输入通道, 32个输出通道,  kernel_size=3
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        # 第二层卷积: 32个输入通道, 64个输出通道, kernel_size=3
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # 池化层: kernel_size=2, stride=2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # 全连接层: 7*7*64 -> 128
        self.fc1 = nn.Linear(7*7*64, 128)
        # 全连接层: 128 -> 10 (10个类别)
        self.fc2 = nn.Linear(128, 10)
        # Dropout层
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # 第一层卷积 + ReLU + 池化
        x = self.pool(F.relu(self.conv1(x)))
        # 第二层卷积 + ReLU + 池化
        x = self.pool(F.relu(self.conv2(x)))
        # 展平
        x = x.view(-1, 7*7*64)
        # 全连接层 + ReLU + Dropout
        x = self.dropout(F.relu(self.fc1(x)))
        # 输出层
        x = self.fc2(x)
        return x

# 检查 mps 是否可用
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS 可用，将使用 MPS 设备")
else:
    print("MPS 不可用，将使用默认设备")
    device = torch.device("cpu")

# 初始化模型并创建在指定设备中
model = SimpleCNN().to(device)

# 加载模型权重
try:
    # 直接使用 mps 加载模型权重
    model.load_state_dict(torch.load('model/mnist_cnn_model.pth', map_location=device))
    model.eval()  # 设置为评估模式
    print("模型加载成功！")
except FileNotFoundError:
    print("未找到 model/mnist_cnn_model.pth 文件，请检查文件路径。")
except Exception as e:
    print(f"加载模型时出错: {e}")

# 导入 MNIST 测试集
# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 加载测试集
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# # 获取一个测试样本
# data, _ = next(iter(test_loader))
# data = data.to(device)

# with torch.no_grad():
#     output = model(data)
#     predicted_class = torch.argmax(output, dim=1)
#     print(f"预测的类别是: {predicted_class.item()}")


# 加载测试图像并显示
def visualize_prediction(model, device, test_loader, num_samples=5):
    model.eval()
    images, labels = next(iter(test_loader))
    images, labels = images[:num_samples].to(device), labels[:num_samples].to(device)
    
    with torch.no_grad():
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
    
    fig, axes = plt.subplots(1, num_samples, figsize=(12, 3))
    for i in range(num_samples):
        ax = axes[i]
        ax.imshow(images[i].cpu().numpy().squeeze(), cmap='gray')
        ax.set_title(f'真实: {labels[i].item()}, 预测: {predicted[i].item()}')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# 可视化预测结果
visualize_prediction(model, device, test_loader)


In [ ]:
import matplotlib as mpl
print(mpl.get_cachedir())
print(mpl.matplotlib_fname())